# Quantitative Value Strategy
"Value investing" means investing in the stocks that are cheapest relative to common measures of business value (like earnings or assets).

For this project, we're going to build an investing strategy that selects the 50 stocks with the best value metrics. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.

## Library Imports
The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [ ]:
! pip install yfinance numpy pandas scipy XlsxWriter ipython

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 159.9/159.9 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 39.6 MB/s eta 0:00:00


In [ ]:
import yfinance as yf
import numpy as np
import pandas as pd
import math
from scipy.stats import percentileofscore as score
from statistics import mean
import xlsxwriter
from IPython.display import clear_output

## Importing Our List of Stocks

The next thing we need to do is import the constituents of the Nifty 50.

These constituents change over time, so in an ideal world you would connect directly to the index provider (Standard & Poor's) and pull their real-time constituents on a regular basis. But, here we have just taken a list of ~50 stocks in a CSV file taken at the time of building the project.

Now it's time to import these stocks to our Jupyter Notebook file.

In [ ]:
stocks = pd.read_csv('/content/ind_nifty50list.csv')
stocks.rename(columns={'Symbol': 'Ticker'}, inplace=True)
stocks['Ticker'] = stocks['Ticker'] + '.NS'
stocks

,Company Name,Industry,Ticker,Series,ISIN Code
0,Adani Enterprises Ltd.,Metals & Mining,ADANIENT.NS,EQ,INE423A01024
1,Adani Ports and Special Economic Zone Ltd.,Services,ADANIPORTS.NS,EQ,INE742F01042
2,Apollo Hospitals Enterprise Ltd.,Healthcare,APOLLOHOSP.NS,EQ,INE437A01024
3,Asian Paints Ltd.,Consumer Durables,ASIANPAINT.NS,EQ,INE021A01026
4,Axis Bank Ltd.,Financial Services,AXISBANK.NS,EQ,INE238A01034
5,Bajaj Auto Ltd.,Automobile and Auto Components,BAJAJ-AUTO.NS,EQ,INE917I01010
6,Bajaj Finance Ltd.,Financial Services,BAJFINANCE.NS,EQ,INE296A01024
7,Bajaj Finserv Ltd.,Financial Services,BAJAJFINSV.NS,EQ,INE918I01026
8,Bharat Electronics Ltd.,Capital Goods,BEL.NS,EQ,INE263A01024
9,Bharat Petroleum Corporation Ltd.,Oil Gas & Consumable Fuels,BPCL.NS,EQ,INE029A01011


## Adding Our Stocks Data to a Pandas DataFrame

The next thing we need to do is add our stock's price and market capitalization to a pandas DataFrame.

In [ ]:
my_columns = ['Ticker',
              'Stock Price',
              'P/E Ratio',
              'Number of Shares to Buy']
final_dataframe = pd.DataFrame(columns=my_columns)
final_dataframe

,Ticker,Stock Price,P/E Ratio,Number of Shares to Buy


## Looping Through The Tickers in Our List of Stocks

Here to fetch the stock data, we have used `yfinance` API

We can pull data for all Nifty 50 stocks and store their data in the DataFrame using a `for` loop.

In [16]:
import time
import numpy as np
import pandas as pd
import yfinance as yf
from IPython.display import clear_output

final_dataframe = pd.DataFrame()

for stock in stocks['Ticker']:
    retries = 3  # Number of retries
    for attempt in range(retries):
        try:
            data = yf.Ticker(stock).info
            price = data['currentPrice']
            try:
                peRatio = data['currentPrice'] / data['trailingEps']
            except:
                peRatio = np.NaN
            stock_dict = {
                'Ticker': [stock],
                'Stock Price': [price],
                'P/E Ratio': [peRatio],
                'Number of Shares to Buy': ['N/A']
            }
            df = pd.DataFrame(stock_dict)
            final_dataframe = pd.concat([final_dataframe, df], ignore_index=True)

            clear_output(wait=True)
            print(final_dataframe)
            break  # Break if the data is successfully retrieved
        except Exception as e:
            print(f"Error fetching data for {stock}. Attempt {attempt + 1}/{retries}")
            if attempt < retries - 1:
                time.sleep(5)  # Wait 5 seconds before retrying
            else:
                print(f"Failed to fetch data for {stock} after {retries} attempts")
                break  # Break after retries exhausted


           Ticker  Stock Price   P/E Ratio Number of Shares to Buy
0     ADANIENT.NS      2564.60   55.247738                     N/A
1   ADANIPORTS.NS      1199.55   25.253684                     N/A
2   APOLLOHOSP.NS      7298.35   88.497029                     N/A
3   ASIANPAINT.NS      2335.95   49.023085                     N/A
4     AXISBANK.NS      1084.90   12.077257                     N/A
5   BAJAJ-AUTO.NS      8965.70   34.129045                     N/A
6   BAJFINANCE.NS      7407.25   29.762335                     N/A
7   BAJAJFINSV.NS      1701.10   32.156900                     N/A
8          BEL.NS       291.95   46.937299                     N/A
9         BPCL.NS       296.40    9.651579                     N/A
10  BHARTIARTL.NS      1598.85   77.501212                     N/A
11   BRITANNIA.NS      4834.10   54.647298                     N/A
12       CIPLA.NS      1511.25   27.323269                     N/A
13   COALINDIA.NS       393.65    6.724462                    

In [17]:
final_dataframe

,Ticker,Stock Price,P/E Ratio,Number of Shares to Buy
0,ADANIENT.NS,2564.60,55.247738,N/A
1,ADANIPORTS.NS,1199.55,25.253684,N/A
2,APOLLOHOSP.NS,7298.35,88.497029,N/A
3,ASIANPAINT.NS,2335.95,49.023085,N/A
4,AXISBANK.NS,1084.90,12.077257,N/A
5,BAJAJ-AUTO.NS,8965.70,34.129045,N/A
6,BAJFINANCE.NS,7407.25,29.762335,N/A
7,BAJAJFINSV.NS,1701.10,32.156900,N/A
8,BEL.NS,291.95,46.937299,N/A
9,BPCL.NS,296.40,9.651579,N/A


In [18]:
final_dataframe[final_dataframe.isnull().any(axis=1)]

,Ticker,Stock Price,P/E Ratio,Number of Shares to Buy


Filling the `NULL` cells of the `P/E Ratio` column with mean value

In [19]:
final_dataframe['P/E Ratio'].fillna(final_dataframe['P/E Ratio'].mean(), inplace=True)

<ipython-input-19-ec01d35b2dc7>:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  final_dataframe['P/E Ratio'].fillna(final_dataframe['P/E Ratio'].mean(), inplace=True)


In [20]:
final_dataframe[final_dataframe.isnull().any(axis=1)]

,Ticker,Stock Price,P/E Ratio,Number of Shares to Buy


In [21]:
final_dataframe

,Ticker,Stock Price,P/E Ratio,Number of Shares to Buy
0,ADANIENT.NS,2564.60,55.247738,N/A
1,ADANIPORTS.NS,1199.55,25.253684,N/A
2,APOLLOHOSP.NS,7298.35,88.497029,N/A
3,ASIANPAINT.NS,2335.95,49.023085,N/A
4,AXISBANK.NS,1084.90,12.077257,N/A
5,BAJAJ-AUTO.NS,8965.70,34.129045,N/A
6,BAJFINANCE.NS,7407.25,29.762335,N/A
7,BAJAJFINSV.NS,1701.10,32.156900,N/A
8,BEL.NS,291.95,46.937299,N/A
9,BPCL.NS,296.40,9.651579,N/A


## Removing Glamour Stocks

The opposite of a "value stock" is a "glamour stock".

Since the goal of this strategy is to identify the 50 best value stocks from our universe, our next step is to remove glamour stocks from the DataFrame.

We'll consider only those stocks whose `P/E Ratio` is positive.

In [22]:
final_dataframe.sort_values('P/E Ratio', ascending=True, inplace=True)
final_dataframe = final_dataframe[final_dataframe['P/E Ratio'] > 0]

In [23]:
final_dataframe

,Ticker,Stock Price,P/E Ratio,Number of Shares to Buy
13,COALINDIA.NS,393.65,6.724462,N/A
34,ONGC.NS,258.89,7.924395,N/A
43,TATAMOTORS.NS,790.40,8.779296,N/A
25,INDUSINDBK.NS,997.90,9.541977,N/A
9,BPCL.NS,296.40,9.651579,N/A
39,SBIN.NS,793.40,9.891535,N/A
21,HINDALCO.NS,591.15,10.537433,N/A
4,AXISBANK.NS,1084.90,12.077257,N/A
38,SHRIRAMFIN.NS,3048.35,14.367488,N/A
28,KOTAKBANK.NS,1838.65,16.750023,N/A


In [24]:
final_dataframe = final_dataframe[:50]
final_dataframe.reset_index(drop=True, inplace=True)

In [25]:
final_dataframe

,Ticker,Stock Price,P/E Ratio,Number of Shares to Buy
0,COALINDIA.NS,393.65,6.724462,N/A
1,ONGC.NS,258.89,7.924395,N/A
2,TATAMOTORS.NS,790.40,8.779296,N/A
3,INDUSINDBK.NS,997.90,9.541977,N/A
4,BPCL.NS,296.40,9.651579,N/A
5,SBIN.NS,793.40,9.891535,N/A
6,HINDALCO.NS,591.15,10.537433,N/A
7,AXISBANK.NS,1084.90,12.077257,N/A
8,SHRIRAMFIN.NS,3048.35,14.367488,N/A
9,KOTAKBANK.NS,1838.65,16.750023,N/A


## Calculating the Number of Shares to Buy

As we can see in the DataFrame above, we stil haven't calculated the number of shares of each stock to buy.

In [26]:
# portfolio_size = float(input("Enter your portfolio value: "))
portfolio_size = 10000000
portfolio_size

10000000

In [27]:
position_size = float(portfolio_size) / len(final_dataframe.index)
position_size

200000.0

In [28]:
for i in range(len(final_dataframe.index)):
    final_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(
        position_size / final_dataframe.loc[i, 'Stock Price'])

final_dataframe

,Ticker,Stock Price,P/E Ratio,Number of Shares to Buy
0,COALINDIA.NS,393.65,6.724462,508
1,ONGC.NS,258.89,7.924395,772
2,TATAMOTORS.NS,790.40,8.779296,253
3,INDUSINDBK.NS,997.90,9.541977,200
4,BPCL.NS,296.40,9.651579,674
5,SBIN.NS,793.40,9.891535,252
6,HINDALCO.NS,591.15,10.537433,338
7,AXISBANK.NS,1084.90,12.077257,184
8,SHRIRAMFIN.NS,3048.35,14.367488,65
9,KOTAKBANK.NS,1838.65,16.750023,108


## Building a Better (and More Realistic) Value Strategy
Every valuation metric has certain flaws.

For example, the price-to-earnings ratio doesn't work well with stocks with negative earnings.

Similarly, stocks that buyback their own shares are difficult to value using the price-to-book ratio.

Investors typically use a `composite` basket of valuation metrics to build robust quantitative value strategies. In this section, we will filter for stocks with the lowest percentiles on the following metrics:

* Price-to-earnings ratio
* Price-to-book ratio
* Price-to-sales ratio
* Enterprise Value divided by Earnings Before Interest, Taxes, Depreciation, and Amortization (EV/EBITDA)
* Enterprise Value divided by Gross Profit (EV/GP)

In [29]:
rv_columns = ['Ticker',
              'Stock Price',
              'Number of Shares to Buy',
              'Price-to-Earning Ratio',
              'PE Percentile',
              'Price-to-Book Value',
              'PB Percentile',
              'Price-to-Sales Ratio',
              'PS Percentile',
              'EV/EBITDA',
              'EV/EBITDA Percentile',
              'EV/GP',
              'EV/GP Percentile',
              'RV Score']

rv_dataframe = pd.DataFrame(columns=rv_columns)
rv_dataframe

,Ticker,Stock Price,Number of Shares to Buy,Price-to-Earning Ratio,PE Percentile,Price-to-Book Value,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score


In [31]:
import time
import numpy as np
import pandas as pd
import yfinance as yf
from IPython.display import clear_output

rv_dataframe = pd.DataFrame()

for stock in stocks['Ticker']:
    retries = 3  # Number of retries
    for attempt in range(retries):
        try:
            data = yf.Ticker(stock).info
            financials = yf.Ticker(stock).financials

            try:
                price = data['currentPrice']
            except:
                price = np.NaN

            try:
                eps = data['trailingEps']
            except:
                eps = 1

            try:
                peRatio = price / eps
            except:
                peRatio = np.NaN

            try:
                pbRatio = data['priceToBook']
            except:
                pbRatio = np.NaN

            try:
                marketCap = data['marketCap']
            except:
                marketCap = np.NaN

            try:
                outstandingShares = data['sharesOutstanding']
            except:
                outstandingShares = 1

            try:
                psRatio = marketCap / (outstandingShares * eps)
            except:
                psRatio = np.NaN

            try:
                debt = data['totalDebt']
            except:
                debt = np.NaN

            try:
                cash = data['totalCash']
            except:
                cash = np.NaN

            try:
                ebitda = data['ebitda']
            except:
                ebitda = 1

            try:
                evToEBIDTA = (marketCap + debt - cash) / ebitda
            except:
                evToEBIDTA = np.NaN

            try:
                grossProfit = financials.loc['Gross Profit'][0]
            except:
                grossProfit = 1

            try:
                evToGP = (marketCap + debt - cash) / grossProfit
            except:
                evToGP = np.NaN

            stock_dict = {
                'Ticker': [stock],
                'Stock Price': [price],
                'Number of Shares to Buy': ['N/A'],
                'Price-to-Earning Ratio': [peRatio],
                'PE Percentile': ['N/A'],
                'Price-to-Book Value': [pbRatio],
                'PB Percentile': ['N/A'],
                'Price-to-Sales Ratio': [psRatio],
                'PS Percentile': ['N/A'],
                'EV/EBITDA': [evToEBIDTA],
                'EV/EBITDA Percentile': ['N/A'],
                'EV/GP': [evToGP],
                'EV/GP Percentile': ['N/A'],
                'RV Score': ['N/A']
            }
            df = pd.DataFrame(stock_dict)
            rv_dataframe = pd.concat([rv_dataframe, df], ignore_index=True)

            clear_output(wait=True)
            print(rv_dataframe)
            break  # Break if the data is successfully retrieved
        except Exception as e:
            print(f"Error fetching data for {stock}. Attempt {attempt + 1}/{retries}")
            if attempt < retries - 1:
                time.sleep(2)  # Wait 2 seconds before retrying
            else:
                print(f"Failed to fetch data for {stock} after {retries} attempts")
                break  # Break after retries exhausted

           Ticker  Stock Price Number of Shares to Buy  \
0     ADANIENT.NS      2564.60                     N/A   
1   ADANIPORTS.NS      1199.55                     N/A   
2   APOLLOHOSP.NS      7298.35                     N/A   
3   ASIANPAINT.NS      2335.95                     N/A   
4     AXISBANK.NS      1084.90                     N/A   
5   BAJAJ-AUTO.NS      8965.70                     N/A   
6   BAJFINANCE.NS      7407.25                     N/A   
7   BAJAJFINSV.NS      1701.10                     N/A   
8          BEL.NS       291.95                     N/A   
9         BPCL.NS       296.40                     N/A   
10  BHARTIARTL.NS      1598.85                     N/A   
11   BRITANNIA.NS      4834.10                     N/A   
12       CIPLA.NS      1511.25                     N/A   
13   COALINDIA.NS       393.65                     N/A   
14     DRREDDY.NS      1352.65                     N/A   
15    DUMMYITC.NS          NaN                     N/A   
16   EICHERMOT

In [32]:
rv_dataframe

,Ticker,Stock Price,Number of Shares to Buy,Price-to-Earning Ratio,PE Percentile,Price-to-Book Value,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,ADANIENT.NS,2564.60,N/A,55.247738,N/A,7.674974,N/A,55.247739,N/A,2.836501e+01,N/A,1.084174e+01,N/A,N/A
1,ADANIPORTS.NS,1199.55,N/A,25.253684,N/A,4.531409,N/A,25.253686,N/A,1.816137e+01,N/A,1.962913e+01,N/A,N/A
2,APOLLOHOSP.NS,7298.35,N/A,88.497029,N/A,14.616725,N/A,88.497032,N/A,4.264189e+01,N/A,1.207955e+01,N/A,N/A
3,ASIANPAINT.NS,2335.95,N/A,49.023085,N/A,12.417604,N/A,49.023084,N/A,3.536065e+01,N/A,1.469225e+01,N/A,N/A
4,AXISBANK.NS,1084.90,N/A,12.077257,N/A,1.945262,N/A,12.077257,N/A,4.717932e+12,N/A,4.717932e+12,N/A,N/A
5,BAJAJ-AUTO.NS,8965.70,N/A,34.129045,N/A,8.067686,N/A,34.129782,N/A,2.219960e+01,N/A,2.174675e+01,N/A,N/A
6,BAJFINANCE.NS,7407.25,N/A,29.762335,N/A,5.278590,N/A,29.779371,N/A,7.041325e+12,N/A,7.041325e+12,N/A,N/A
7,BAJAJFINSV.NS,1701.10,N/A,32.156900,N/A,3.966387,N/A,32.156900,N/A,1.136469e+01,N/A,5.107235e+12,N/A,N/A
8,BEL.NS,291.95,N/A,46.937299,N/A,12.039672,N/A,46.937300,N/A,3.551883e+01,N/A,2.199829e+01,N/A,N/A
9,BPCL.NS,296.40,N/A,9.651579,N/A,1.641350,N/A,9.651579,N/A,7.016481e+00,N/A,3.018774e+00,N/A,N/A


In [33]:
rv_dataframe[rv_dataframe.isnull().any(axis=1)]

,Ticker,Stock Price,Number of Shares to Buy,Price-to-Earning Ratio,PE Percentile,Price-to-Book Value,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
15,DUMMYITC.NS,NaN,N/A,NaN,N/A,NaN,N/A,NaN,N/A,NaN,N/A,NaN,N/A,N/A
38,SBILIFE.NS,1447.7,N/A,66.652855,N/A,8.911006,N/A,66.652852,N/A,NaN,N/A,NaN,N/A,N/A


## Dealing With Missing Data in Our DataFrame

Our DataFrame contains some missing data which we have to fill with `mean` values.

In [34]:
for column in ['Price-to-Earning Ratio', 'Price-to-Book Value', 'Price-to-Sales Ratio', 'EV/EBITDA', 'EV/GP']:
    rv_dataframe[column].fillna(rv_dataframe[column].mean(), inplace=True)

<ipython-input-34-8c83ea21c654>:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  rv_dataframe[column].fillna(rv_dataframe[column].mean(), inplace=True)


In [35]:
rv_dataframe[rv_dataframe.isnull().any(axis=1)]

,Ticker,Stock Price,Number of Shares to Buy,Price-to-Earning Ratio,PE Percentile,Price-to-Book Value,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
15,DUMMYITC.NS,NaN,N/A,38.24474,N/A,35.829713,N/A,38.342389,N/A,1.132528e+12,N/A,1.263950e+12,N/A,N/A


## Calculating Value Percentiles

We now need to calculate value score percentiles for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* Price-to-earnings ratio
* Price-to-book ratio
* Price-to-sales ratio
* EV/EBITDA
* EV/GP

In [36]:
metrics = {
    'Price-to-Earning Ratio': 'PE Percentile',
    'Price-to-Book Value': 'PB Percentile',
    'Price-to-Sales Ratio': 'PS Percentile',
    'EV/EBITDA': 'EV/EBITDA Percentile',
    'EV/GP': 'EV/GP Percentile'
}

for metric in metrics.keys():
    for row in rv_dataframe.index:
        rv_dataframe.loc[row, metrics[metric]] = score(rv_dataframe[metric], rv_dataframe.loc[row, metric])/100

In [37]:
rv_dataframe

,Ticker,Stock Price,Number of Shares to Buy,Price-to-Earning Ratio,PE Percentile,Price-to-Book Value,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,ADANIENT.NS,2564.60,N/A,55.247738,0.823529,7.674974,0.666667,55.247739,0.823529,2.836501e+01,0.529412,1.084174e+01,0.372549,N/A
1,ADANIPORTS.NS,1199.55,N/A,25.253684,0.352941,4.531409,0.470588,25.253686,0.352941,1.816137e+01,0.372549,1.962913e+01,0.627451,N/A
2,APOLLOHOSP.NS,7298.35,N/A,88.497029,0.960784,14.616725,0.862745,88.497032,0.960784,4.264189e+01,0.686275,1.207955e+01,0.470588,N/A
3,ASIANPAINT.NS,2335.95,N/A,49.023085,0.745098,12.417604,0.823529,49.023084,0.745098,3.536065e+01,0.588235,1.469225e+01,0.54902,N/A
4,AXISBANK.NS,1084.90,N/A,12.077257,0.156863,1.945262,0.176471,12.077257,0.156863,4.717932e+12,0.921569,4.717932e+12,0.901961,N/A
5,BAJAJ-AUTO.NS,8965.70,N/A,34.129045,0.568627,8.067686,0.705882,34.129782,0.568627,2.219960e+01,0.470588,2.174675e+01,0.666667,N/A
6,BAJFINANCE.NS,7407.25,N/A,29.762335,0.470588,5.278590,0.54902,29.779371,0.470588,7.041325e+12,0.941176,7.041325e+12,0.941176,N/A
7,BAJAJFINSV.NS,1701.10,N/A,32.156900,0.54902,3.966387,0.411765,32.156900,0.54902,1.136469e+01,0.196078,5.107235e+12,0.921569,N/A
8,BEL.NS,291.95,N/A,46.937299,0.72549,12.039672,0.803922,46.937300,0.72549,3.551883e+01,0.607843,2.199829e+01,0.686275,N/A
9,BPCL.NS,296.40,N/A,9.651579,0.098039,1.641350,0.098039,9.651579,0.098039,7.016481e+00,0.078431,3.018774e+00,0.098039,N/A


## Calculating the RV Score
We'll now calculate our RV Score (which stands for Robust Value), which is the value score that we'll use to filter for stocks in this investing strategy.

The RV Score will be the arithmetic mean of the 4 percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the mean function from Python's built-in statistics module.

In [38]:
for row in rv_dataframe.index:
    value_percentiles = []
    for metric in metrics.keys():
        value_percentiles.append(rv_dataframe.loc[row, metrics[metric]])
    rv_dataframe.loc[row, 'RV Score'] = mean(value_percentiles)

In [39]:
rv_dataframe

,Ticker,Stock Price,Number of Shares to Buy,Price-to-Earning Ratio,PE Percentile,Price-to-Book Value,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,ADANIENT.NS,2564.60,N/A,55.247738,0.823529,7.674974,0.666667,55.247739,0.823529,2.836501e+01,0.529412,1.084174e+01,0.372549,0.643137
1,ADANIPORTS.NS,1199.55,N/A,25.253684,0.352941,4.531409,0.470588,25.253686,0.352941,1.816137e+01,0.372549,1.962913e+01,0.627451,0.435294
2,APOLLOHOSP.NS,7298.35,N/A,88.497029,0.960784,14.616725,0.862745,88.497032,0.960784,4.264189e+01,0.686275,1.207955e+01,0.470588,0.788235
3,ASIANPAINT.NS,2335.95,N/A,49.023085,0.745098,12.417604,0.823529,49.023084,0.745098,3.536065e+01,0.588235,1.469225e+01,0.54902,0.690196
4,AXISBANK.NS,1084.90,N/A,12.077257,0.156863,1.945262,0.176471,12.077257,0.156863,4.717932e+12,0.921569,4.717932e+12,0.901961,0.462745
5,BAJAJ-AUTO.NS,8965.70,N/A,34.129045,0.568627,8.067686,0.705882,34.129782,0.568627,2.219960e+01,0.470588,2.174675e+01,0.666667,0.596078
6,BAJFINANCE.NS,7407.25,N/A,29.762335,0.470588,5.278590,0.54902,29.779371,0.470588,7.041325e+12,0.941176,7.041325e+12,0.941176,0.67451
7,BAJAJFINSV.NS,1701.10,N/A,32.156900,0.54902,3.966387,0.411765,32.156900,0.54902,1.136469e+01,0.196078,5.107235e+12,0.921569,0.52549
8,BEL.NS,291.95,N/A,46.937299,0.72549,12.039672,0.803922,46.937300,0.72549,3.551883e+01,0.607843,2.199829e+01,0.686275,0.709804
9,BPCL.NS,296.40,N/A,9.651579,0.098039,1.641350,0.098039,9.651579,0.098039,7.016481e+00,0.078431,3.018774e+00,0.098039,0.094118


## Selecting the 50 Best Value Stocks¶

As before, we can identify the 50 best value stocks in our universe by sorting the DataFrame on the RV Score column and dropping all but the top 50 entries.

In [40]:
rv_dataframe.sort_values('RV Score', ascending=True, inplace=True)

In [41]:
rv_dataframe = rv_dataframe[:50]
rv_dataframe.reset_index(drop=True, inplace=True)

In [42]:
rv_dataframe

,Ticker,Stock Price,Number of Shares to Buy,Price-to-Earning Ratio,PE Percentile,Price-to-Book Value,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,ONGC.NS,258.89,N/A,7.924395,0.039216,0.924030,0.019608,7.924396,0.039216,5.685079e+00,0.039216,3.436704e+00,0.117647,0.05098
1,COALINDIA.NS,393.65,N/A,6.724462,0.019608,2.520634,0.27451,6.724462,0.019608,4.517068e+00,0.019608,1.937021e+00,0.019608,0.070588
2,HINDALCO.NS,591.15,N/A,10.537433,0.137255,1.140606,0.039216,10.537434,0.137255,6.263267e+00,0.058824,2.596483e+00,0.058824,0.086275
3,BPCL.NS,296.40,N/A,9.651579,0.098039,1.641350,0.098039,9.651579,0.098039,7.016481e+00,0.078431,3.018774e+00,0.098039,0.094118
4,TATAMOTORS.NS,790.40,N/A,8.779296,0.058824,2.689854,0.294118,8.779296,0.058824,7.079621e+00,0.098039,2.307916e+00,0.039216,0.109804
5,NTPC.NS,339.85,N/A,17.242516,0.215686,1.958395,0.196078,17.242517,0.215686,1.121633e+01,0.176471,8.560617e+00,0.294118,0.219608
6,POWERGRID.NS,316.05,N/A,18.834923,0.235294,3.191586,0.372549,18.840897,0.235294,1.029980e+01,0.117647,9.630442e+00,0.333333,0.258824
7,RELIANCE.NS,1251.15,N/A,24.938210,0.333333,2.066060,0.215686,24.938210,0.333333,1.174931e+01,0.215686,8.106512e+00,0.27451,0.27451
8,DRREDDY.NS,1352.65,N/A,20.408117,0.294118,3.690330,0.392157,20.409636,0.294118,1.360079e+01,0.254902,6.799356e+00,0.215686,0.290196
9,WIPRO.NS,294.45,N/A,26.313673,0.372549,1.880172,0.117647,26.313674,0.372549,1.660210e+01,0.313725,1.037318e+01,0.352941,0.305882


## Calculating the Number of Shares to Buy

As we can see in the DataFrame above, we stil haven't calculated the number of shares of each stock to buy.

In [43]:
# portfolio_size = float(input("Enter your portfolio value: "))
portfolio_size = 10000000
portfolio_size

10000000

In [44]:
position_size = float(portfolio_size) / len(rv_dataframe.index)
position_size

200000.0

In [45]:
for row in rv_dataframe.index:
    rv_dataframe.loc[row, 'Number of Shares to Buy'] = math.floor(
        position_size / final_dataframe.loc[row, 'Stock Price'])

In [46]:
rv_dataframe

,Ticker,Stock Price,Number of Shares to Buy,Price-to-Earning Ratio,PE Percentile,Price-to-Book Value,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,ONGC.NS,258.89,508,7.924395,0.039216,0.924030,0.019608,7.924396,0.039216,5.685079e+00,0.039216,3.436704e+00,0.117647,0.05098
1,COALINDIA.NS,393.65,772,6.724462,0.019608,2.520634,0.27451,6.724462,0.019608,4.517068e+00,0.019608,1.937021e+00,0.019608,0.070588
2,HINDALCO.NS,591.15,253,10.537433,0.137255,1.140606,0.039216,10.537434,0.137255,6.263267e+00,0.058824,2.596483e+00,0.058824,0.086275
3,BPCL.NS,296.40,200,9.651579,0.098039,1.641350,0.098039,9.651579,0.098039,7.016481e+00,0.078431,3.018774e+00,0.098039,0.094118
4,TATAMOTORS.NS,790.40,674,8.779296,0.058824,2.689854,0.294118,8.779296,0.058824,7.079621e+00,0.098039,2.307916e+00,0.039216,0.109804
5,NTPC.NS,339.85,252,17.242516,0.215686,1.958395,0.196078,17.242517,0.215686,1.121633e+01,0.176471,8.560617e+00,0.294118,0.219608
6,POWERGRID.NS,316.05,338,18.834923,0.235294,3.191586,0.372549,18.840897,0.235294,1.029980e+01,0.117647,9.630442e+00,0.333333,0.258824
7,RELIANCE.NS,1251.15,184,24.938210,0.333333,2.066060,0.215686,24.938210,0.333333,1.174931e+01,0.215686,8.106512e+00,0.27451,0.27451
8,DRREDDY.NS,1352.65,65,20.408117,0.294118,3.690330,0.392157,20.409636,0.294118,1.360079e+01,0.254902,6.799356e+00,0.215686,0.290196
9,WIPRO.NS,294.45,108,26.313673,0.372549,1.880172,0.117647,26.313674,0.372549,1.660210e+01,0.313725,1.037318e+01,0.352941,0.305882


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

### Initializing our XlsxWriter Object

In [47]:
writer = pd.ExcelWriter('value_strategy.xlsx', engine='xlsxwriter')
rv_dataframe.to_excel(writer, sheet_name='Value Strategy', index=False)

## Formatting Our Excel Output

In [48]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
    {
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
    }
)

rupee_template = writer.book.add_format(
    {
        'num_format': '₹0.00',
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
    }
)

integer_template = writer.book.add_format(
    {
        'num_format': '0',
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
    }
)

float_template = writer.book.add_format(
    {
        'num_format': '0.0',
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
    }
)

percent_template = writer.book.add_format(
    {
        'num_format': '0.0%',
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
    }
)

In [49]:
column_formats = {
    'A': ['Ticker', string_template],
    'B': ['Stock Price', rupee_template],
    'C': ['Number of Shares to Buy', integer_template],
    'D': ['Price-to-Earning Ratio', float_template],
    'E': ['PE Percentile', percent_template],
    'F': ['Price-to-Book Value', float_template],
    'G': ['PB Percentile', percent_template],
    'H': ['Price-to-Sales Ratio', float_template],
    'I': ['PS Percentile', percent_template],
    'J': ['EV/EBITDA', float_template],
    'K': ['EV/EBITDA Percentile', percent_template],
    'L': ['EV/GP', float_template],
    'M': ['EV/GP Percentile', percent_template],
    'N': ['RV Score', percent_template]
}

for column in column_formats.keys():
    writer.sheets['Value Strategy'].set_column(f'{column}:{column}', 20, column_formats[column][1])
    writer.sheets['Value Strategy'].write(f'{column}1', column_formats[column][0], string_template)

## Saving Our Excel Output

In [50]:
writer.close()